# 🎯 Pothole Detection - YOLO Baseline (No Preprocessing)

Notebook ini adalah **versi baseline** untuk perbandingan dengan YOLO + Preprocessing.

**Karakteristik:**
- ❌ Tidak ada preprocessing (CLAHE, sharpening, bilateral filter)
- ❌ Tidak ada augmentasi data
- ✅ Training langsung dengan dataset original
- ✅ Arsitektur YOLO yang sama
- ✅ Hyperparameter yang sama

**Tujuan:** Mengukur dampak preprocessing terhadap performa model

## 📦 Step 1: Setup & Installation

In [ ]:
# Install required packages
!pip install ultralytics opencv-python-headless matplotlib Pillow pyyaml

In [ ]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import shutil
import yaml
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
import torch

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 📂 Step 2: Dataset Configuration

⚠️ **IMPORTANT:** Sesuaikan path berikut dengan lokasi dataset Anda!

In [ ]:
# ============================================
# KONFIGURASI PATH - SESUAIKAN!
# ============================================

# Path ke dataset ORIGINAL (tanpa preprocessing)
ORIGINAL_DATASET = '/content/pothole_dataset'  # Ganti dengan path dataset Anda

# Output directory untuk hasil training
OUTPUT_DIR = '/content/yolo_baseline_output'

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Original Dataset: {ORIGINAL_DATASET}")
print(f"📁 Output Directory: {OUTPUT_DIR}")

## 🔍 Step 3: Inspect Dataset

Verifikasi struktur dan jumlah data

In [ ]:
def inspect_dataset(dataset_path):
    """
    Inspect dataset structure and count files
    """
    splits = ['train', 'valid', 'test']
    total_images = 0
    total_labels = 0
    
    print("\n" + "="*60)
    print("📊 DATASET INSPECTION")
    print("="*60)
    
    for split in splits:
        images_path = os.path.join(dataset_path, split, 'images')
        labels_path = os.path.join(dataset_path, split, 'labels')
        
        if os.path.exists(images_path):
            num_images = len(glob.glob(os.path.join(images_path, '*.*')))
            num_labels = len(glob.glob(os.path.join(labels_path, '*.txt')))
            
            total_images += num_images
            total_labels += num_labels
            
            print(f"\n{split.upper()}:")
            print(f"  📷 Images: {num_images}")
            print(f"  🏷️  Labels: {num_labels}")
        else:
            print(f"\n{split.upper()}: ❌ Not found")
    
    print(f"\n{'='*60}")
    print(f"📊 TOTAL: {total_images} images, {total_labels} labels")
    print(f"{'='*60}\n")
    
    return total_images

# Inspect dataset
total_data = inspect_dataset(ORIGINAL_DATASET)

## 🖼️ Step 4: Visualize Sample Images

Tampilkan beberapa sampel gambar dari dataset original (tanpa preprocessing)

In [ ]:
def visualize_sample_with_bbox(dataset_path, num_samples=3):
    """
    Visualize sample images with bounding boxes
    """
    images_path = os.path.join(dataset_path, 'train', 'images')
    labels_path = os.path.join(dataset_path, 'train', 'labels')
    
    image_files = glob.glob(os.path.join(images_path, '*.*'))[:num_samples]
    
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    if num_samples == 1:
        axes = [axes]
    
    for idx, img_path in enumerate(image_files):
        # Read image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        
        # Read corresponding label
        img_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(labels_path, f"{img_name}.txt")
        
        # Draw bounding boxes
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # YOLO format: class_id x_center y_center width height (normalized)
                        x_center, y_center, width, height = map(float, parts[1:5])
                        
                        # Convert to pixel coordinates
                        x1 = int((x_center - width/2) * w)
                        y1 = int((y_center - height/2) * h)
                        x2 = int((x_center + width/2) * w)
                        y2 = int((y_center + height/2) * h)
                        
                        # Draw rectangle
                        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(image, 'pothole', (x1, y1-10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        axes[idx].imshow(image)
        axes[idx].axis('off')
        axes[idx].set_title(f'Sample {idx+1} (Original)', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'sample_original_images.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Sample visualization saved!")

# Visualize samples
visualize_sample_with_bbox(ORIGINAL_DATASET, num_samples=3)

## 📝 Step 5: Create data.yaml for YOLO

File konfigurasi untuk training YOLO

In [ ]:
# Create data.yaml
data_yaml_path = os.path.join(ORIGINAL_DATASET, 'data.yaml')

data_yaml_content = f"""
# Dataset paths
train: {os.path.join(ORIGINAL_DATASET, 'train', 'images')}
val: {os.path.join(ORIGINAL_DATASET, 'valid', 'images')}
test: {os.path.join(ORIGINAL_DATASET, 'test', 'images')}

# Number of classes
nc: 1

# Class names
names: ['pothole']
"""

with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content.strip())

print(f"✅ data.yaml created at: {data_yaml_path}")
print("\nContent:")
print(data_yaml_content)

---

# 🚀 TRAINING PHASE

## 🎯 Step 6: Initialize YOLO Model

In [ ]:
# Initialize YOLOv8 model
print("Initializing YOLOv8 model...")
model = YOLO('yolov8n.pt')  # nano model for faster training

print("✅ Model initialized successfully!")
print(f"\nModel Summary:")
print(model.model)

## 🔧 Step 7: Training Configuration

**Hyperparameters** (sama dengan versi preprocessing untuk perbandingan yang fair):

In [ ]:
# Check CUDA availability
print(f"torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Training hyperparameters
EPOCHS = 50
BATCH_SIZE = 4  # Sesuaikan dengan kapasitas GPU/RAM Anda
IMG_SIZE = 640
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print("\n" + "="*60)
print("⚙️ TRAINING CONFIGURATION")
print("="*60)
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Image Size: {IMG_SIZE}")
print(f"Device: {DEVICE}")
print("="*60)

## 🏋️ Step 8: Train Model

⚠️ **CATATAN:** Training akan memakan waktu tergantung hardware Anda.
- GPU: ~30-60 menit
- CPU: Beberapa jam

In [ ]:
# Train the model
print("🚀 Starting training...\n")

results = model.train(
    data=data_yaml_path,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    project='runs/detect',
    name='pothole_yolo_baseline',
    save=True,
    save_period=10,  # Save checkpoint every 10 epochs
    patience=15,  # Early stopping patience
    verbose=True,
    plots=True
)

print("\n✅ Training completed!")
print(f"Results saved in: runs/detect/pothole_yolo_baseline")

---

# 📊 EVALUATION PHASE

## 📈 Step 9: Visualize Training Results

In [ ]:
# Load training results
results_dir = 'runs/detect/pothole_yolo_baseline'

# Plot training curves
results_img = os.path.join(results_dir, 'results.png')
if os.path.exists(results_img):
    img = Image.open(results_img)
    plt.figure(figsize=(15, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Training Results - Baseline (No Preprocessing)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves_baseline.png'), dpi=150, bbox_inches='tight')
    plt.show()
else:
    print(f"❌ Results image not found at: {results_img}")

## 🎯 Step 10: Load Best Model & Evaluate on Test Set

In [ ]:
# Load best model
best_model_path = os.path.join(results_dir, 'weights', 'best.pt')
best_model = YOLO(best_model_path)

print(f"✅ Best model loaded from: {best_model_path}")

# Validate on test set
print("\n🔍 Evaluating on test set...\n")
metrics = best_model.val(
    data=data_yaml_path,
    split='test',
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    plots=True
)

# Extract metrics
print("\n" + "="*60)
print("📊 TEST SET PERFORMANCE - BASELINE (NO PREPROCESSING)")
print("="*60)
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print("="*60)

## 🖼️ Step 11: Visualize Predictions on Test Images

In [ ]:
# Predict on random test images
test_images = glob.glob(os.path.join(ORIGINAL_DATASET, 'test', 'images', '*.*'))
sample_images = np.random.choice(test_images, size=min(6, len(test_images)), replace=False)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, img_path in enumerate(sample_images):
    # Predict
    results = best_model.predict(img_path, conf=0.25, device=DEVICE)
    
    # Get annotated image
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(annotated)
    axes[idx].axis('off')
    axes[idx].set_title(f'Test Image {idx+1}', fontsize=10)

plt.suptitle('Predictions - Baseline Model (No Preprocessing)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'test_predictions_baseline.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✅ Predictions visualization saved!")

## 📊 Step 12: Confusion Matrix

In [ ]:
# Load confusion matrix
conf_matrix_img = os.path.join(results_dir, 'confusion_matrix_normalized.png')

if os.path.exists(conf_matrix_img):
    img = Image.open(conf_matrix_img)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix - Baseline (No Preprocessing)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix_baseline.png'), dpi=150, bbox_inches='tight')
    plt.show()
else:
    print(f"❌ Confusion matrix not found at: {conf_matrix_img}")

## 💾 Step 13: Save Results Summary

In [ ]:
# Save metrics to file
summary_path = os.path.join(OUTPUT_DIR, 'baseline_summary.txt')

with open(summary_path, 'w') as f:
    f.write("="*60 + "\n")
    f.write("YOLO BASELINE RESULTS (NO PREPROCESSING)\n")
    f.write("="*60 + "\n\n")
    
    f.write("DATASET INFORMATION:\n")
    f.write(f"- Total Images: {total_data}\n")
    f.write(f"- Preprocessing: None\n")
    f.write(f"- Augmentation: None\n\n")
    
    f.write("TRAINING CONFIGURATION:\n")
    f.write(f"- Model: YOLOv8n\n")
    f.write(f"- Epochs: {EPOCHS}\n")
    f.write(f"- Batch Size: {BATCH_SIZE}\n")
    f.write(f"- Image Size: {IMG_SIZE}\n")
    f.write(f"- Device: {DEVICE}\n\n")
    
    f.write("TEST SET PERFORMANCE:\n")
    f.write(f"- Precision: {metrics.box.mp:.4f}\n")
    f.write(f"- Recall: {metrics.box.mr:.4f}\n")
    f.write(f"- mAP@0.5: {metrics.box.map50:.4f}\n")
    f.write(f"- mAP@0.5:0.95: {metrics.box.map:.4f}\n\n")
    
    f.write("="*60 + "\n")
    f.write("NOTE: This is the baseline model without any preprocessing.\n")
    f.write("Compare with preprocessing results to measure improvement.\n")
    f.write("="*60 + "\n")

print(f"✅ Summary saved to: {summary_path}")

# Display summary
with open(summary_path, 'r') as f:
    print("\n" + f.read())

## 🎬 Step 14: Inference Demo on New Image

In [ ]:
# Demo inference on new image
# Upload gambar baru atau gunakan dari test set

# Example: pilih 1 gambar dari test set
demo_image = test_images[0]

print(f"Testing on: {os.path.basename(demo_image)}")

# Run prediction
results = best_model.predict(
    source=demo_image,
    conf=0.25,
    device=DEVICE,
    save=True,
    project=OUTPUT_DIR,
    name='inference_demo'
)

# Display result
annotated = results[0].plot()
annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(annotated)
plt.axis('off')
plt.title('Inference Demo - Baseline Model', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'inference_demo_baseline.png'), dpi=150, bbox_inches='tight')
plt.show()

# Print detections
print("\nDetections:")
for box in results[0].boxes:
    conf = box.conf.item()
    cls = int(box.cls.item())
    print(f"  - Class: {best_model.names[cls]}, Confidence: {conf:.2f}")

---

# 📊 COMPARISON TEMPLATE

Gunakan cell berikut untuk membandingkan dengan model preprocessing

## 📈 Step 15: Performance Comparison Table

In [ ]:
# Template untuk perbandingan
# Setelah training kedua model selesai, isi nilai-nilai berikut:

print("\n" + "="*80)
print("📊 PERFORMANCE COMPARISON: BASELINE vs PREPROCESSING")
print("="*80)
print()
print(f"{'Metric':<20} {'Baseline (No Prep)':<20} {'With Preprocessing':<20} {'Improvement':<15}")
print("-" * 80)

# Baseline metrics (sudah ada)
baseline_precision = metrics.box.mp
baseline_recall = metrics.box.mr
baseline_map50 = metrics.box.map50
baseline_map = metrics.box.map

# TODO: Isi dengan hasil dari model preprocessing
# prep_precision = 0.XXX  # Ganti dengan nilai aktual
# prep_recall = 0.XXX
# prep_map50 = 0.XXX
# prep_map = 0.XXX

print(f"{'Precision':<20} {baseline_precision:<20.4f} {'TBD':<20} {'TBD':<15}")
print(f"{'Recall':<20} {baseline_recall:<20.4f} {'TBD':<20} {'TBD':<15}")
print(f"{'mAP@0.5':<20} {baseline_map50:<20.4f} {'TBD':<20} {'TBD':<15}")
print(f"{'mAP@0.5:0.95':<20} {baseline_map:<20.4f} {'TBD':<20} {'TBD':<15}")

print("=" * 80)
print("\n⚠️ NOTE: 'TBD' akan diisi setelah training model dengan preprocessing selesai")
print("\n💡 TIP: Copy nilai dari notebook preprocessing dan uncomment code di atas untuk perbandingan lengkap")

## 📝 Step 16: Analysis & Conclusions

In [ ]:
print("""
═══════════════════════════════════════════════════════════════════════════════
📊 BASELINE MODEL ANALYSIS (NO PREPROCESSING)
═══════════════════════════════════════════════════════════════════════════════

✅ STRENGTHS:
   - Faster training time (no preprocessing overhead)
   - Simpler pipeline (fewer moving parts)
   - Lower computational requirements
   - Direct learning from raw data

❌ POTENTIAL WEAKNESSES:
   - May struggle with:
     * Low contrast images
     * Images with varying lighting conditions
     * Noisy images
     * Blurry images
   - Limited data diversity (no augmentation)
   - May require more training data to generalize well

🔍 EXPECTED IMPROVEMENTS WITH PREPROCESSING:
   - Better feature extraction through CLAHE
   - Improved edge detection through sharpening
   - Noise reduction through bilateral filtering
   - Better generalization through augmentation

📈 NEXT STEPS:
   1. Train model WITH preprocessing
   2. Compare metrics side-by-side
   3. Analyze error cases
   4. Determine if preprocessing overhead is worth the improvement

═══════════════════════════════════════════════════════════════════════════════
""")

---

# ✅ CHECKLIST

Setelah menjalankan kedua notebook (baseline & preprocessing), pastikan:

- [ ] Baseline model telah di-training
- [ ] Preprocessing model telah di-training
- [ ] Metrics dari kedua model telah dicatat
- [ ] Visualisasi perbandingan telah dibuat
- [ ] Analisis perbedaan telah dilakukan
- [ ] Kesimpulan tentang efektivitas preprocessing telah dibuat

## 📚 Referensi untuk Thesis:

Hasil dari notebook ini akan menjadi **baseline** untuk menunjukkan improvement yang diberikan oleh preprocessing dalam thesis Anda.

Pastikan untuk:
1. Dokumentasikan semua hyperparameters yang sama
2. Gunakan train/val/test split yang sama
3. Catat waktu training kedua model
4. Bandingkan inference time
5. Analisis kasus-kasus error

---

**Good luck dengan thesis-nya! 🎓🚀**